In [1]:
import os,sys

notebook_dir = os.getcwd()

#add path
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

In [2]:
from scripts.load_data import *
from scripts.data_preparation import *
from scripts.training_and_evaluation import *

KeyboardInterrupt: 

In [ ]:
fra_data_path = '../../Data/Fraud_Data.csv'
# IpAd_data_path = '../../Data/IpAddress_to_Country.csv'
cred_data_path = '../../Data/creditcard.csv/creditcard.csv'

loaded_data_fra = load_data(fra_data_path)
loaded_data_cred = load_data(cred_data_path)

# loaded_data_fra.head(5)
# loaded_data_cred.head(5)

In [ ]:
# Prepare data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = prepare_data(fraud_data)
X_train_cc, X_test_cc, y_train_cc, y_test_cc = prepare_data(creditcard_data)